# Set-valued prediction based on utility maximization 

<div align="justify">


In line with classical statistics, but unlike decision theory and machine learning, the setting of conformal prediction does not involve any notion of loss function. In this regard, it differs from methods for set-valued prediction based on utility maximization (or loss minimization), which are primarily used for multi-class classification problems. Similar to conformal prediction, such methods also return a set of classes when the classifier is too uncertain with respect to the class label to predict, but the interpretation of this set is different. Instead of returning a set that contains the true class label with high probability, sets that maximize a set-based utility score are sought. Besides, being based on the conditional distribution $ \prob(y\,|\,\vec{x}_q) $ of the outcome $ y $ given a query $ \vec{x}_q $, most of these methods capture *conditional* uncertainty. 

Let $u(y,\sety)$ be a set-based utility score, where $y$ denotes the ground truth outcome and $\sety$ the predicted set. Then, adopting a decision-theoretic perspective, the Bayes-optimal solution $\sety^{*}_u$ is found by maximizing the following objective:

$$
\sety^{*}_u(\vec{x}_q) = \argmax_{\sety \in 2^{\mathcal{Y}}\setminus \{\emptyset\}} \evalue_{p(y\,|\,\vec{x}_q)} \big( u(y,\sety) \big) = \argmax_{\sety \in 2^{\mathcal{Y}}\setminus \{\emptyset\}} \sum_{y \in \mathcal{Y}} u(y,\sety) \, p(y\,|\,\vec{x}_q) \,.
$$(bayesoptimal)

</div>

<div align="justify">



Solving {eq}`bayesoptimal` as a brute-force search requires checking all subsets of $\mathcal{Y}$, resulting in an exponential time complexity. However, for many utility scores, the Bayes-optimal prediction can be found more efficiently. Various methods in this direction have been proposed under different names and qualifications of predictions, such as "indeterminate" {cite:t}`zaffalon2002`, "credal" {cite:t}`cora_lr08`, "non-deterministic" {cite:t}`delc_ln09`, and "cautious" {cite:t}`Yang2017b`. Although the methods typically differ in the exact shape of the utility function $u: \mathcal{Y} \times 2^{\mathcal{Y}}\setminus \{\emptyset\} \longrightarrow [0,1]$, most functions are specific members of the following family:

$$

u(y,\sety) = \left \{ 
\begin{array}{cl}
0 &\quad \mbox{if $y \notin \sety$} \ \\
g(|\sety|)&\quad \mbox{if $y \in \sety$} 
\end{array} \, ,
\right.

$$(ufamily)


where $|\sety|$ denotes the cardinality of the predicted set $\sety$. 
This family is characterized  by a sequence $(g(1), \ldots,g(K)) \in [0,1]^K$ with $K$ the number of classes. Ideally, $g$ should obey the following properties: 

1. $g(1) = 1$, i.e., the utility $u(y,\sety)$ should be maximal when the classifier returns the true class label as a singleton set.
2. $g(s)$ should be non-increasing, i.e., the utility $u(y,\sety)$ should be higher if the true class is contained in a smaller set of predicted classes.
3. $g(s) \geq 1/s$, i.e., the utility $u(y,\sety)$ of predicting a set containing the true and $s-1$ additional classes should not be lower than the expected utility of randomly guessing one of these $s$ classes. This requirement formalizes the idea of risk-aversion: in the face of uncertainty, abstaining should be preferred to random guessing {cite:p}`Zaffalon2012EvaluatingCC`. 

Many existing set-based utility scores are recovered as special cases of {eq}`ufamily`, including the three classical measures from information retrieval discussed by {cite:t}`delc_ln09`: precision with $g_P(s) = 1/s$, recall with $g_R(s)=1$, and the F$_{1}$-measure with $g_{F1}(s) = 2/(1+s)$.  Other utility functions with specific choices for $g$ are studied in the literature on credal classification {cite:p}`Corani2008NCC,Corani2009LNCC,Zaffalon2012EvaluatingCC,Yang2017b,mpub385`, such as

$$

g_{\delta,\gamma}(s) := \frac{\delta}{s} - \frac{\gamma}{s^2} \,, \quad g_{\exp}(s) := 1- \exp{\left(-\frac{\delta}{s}\right)},   \quad g_{log}(s) := \log \left(1 + \frac{1}{s} \right) \,.

$$
Especially $g_{\delta,\gamma}(s)$ is commonly used in this community, where $\delta$ and $\gamma$ can only take certain values to guarantee that the utility is in the interval $[0,1]$. Precision (here called discounted accuracy) corresponds to the case $(\delta,\gamma)=(1,0)$. However, typical choices for $(\delta,\gamma$) are $(1.6,0.6)$ and $(2.2,1.2)$ \citep{mpub385}, implementing the idea of risk aversion. The measure $g_{\exp}(s)$ is an exponentiated version of precision, where the parameter $\delta$ also defines the degree of risk aversion. 

</div>


<div align="justify">


Another example appears in the literature on binary or multi-class classification with reject option {cite:p}`herb_cw06,linu_cw18,Ramaswamy2015CAMCRO`. Here, the prediction can only be a singleton or the full set $\mathcal{Y}$ containing $K$ classes. The first case typically gets a reward of one (if the predicted class is correct), while the second case should receive a lower reward, e.g.\ $1-\alpha$.  The latter corresponds to abstaining, i.e., not predicting any class label, and the (user-defined) parameter $\alpha$ specifies a penalty for doing so, with the requirement $0< \alpha < 1-1/K$ to be risk-averse. 

Set-valued predictions are also considered in hierarchical multi-class classification, mostly in the form of internal nodes of the hierarchy  {cite:p}`Freitas_atutorial,Rangwala2017,Yang2017`. Compared to the ''flat'' multi-class case, the prediction space is thus restricted, because only sets of classes that correspond to nodes of the hierarchy can be returned as a prediction. Some of the above utility scores also appear here. For example, \citet{Yang2017} evaluate various members of the $u_{\delta,\gamma}$ family in a framework where hierarchies are considered for computational reasons, while {cite:t}`Oh2017TopKHC` optimizes recall by fixing $|\sety|$ as a user-defined parameter. 
Popular in hierarchical classification is the tree-distance loss, which could also be interpreted as a way of evaluating set-valued predictions {cite:p}`Bi2015`. 
This loss is not a member of the family ({eq}`ufamily`), however. 
Besides, it appears to be a less interesting loss function from the perspective of abstention in cases of uncertainty, since by minimizing the tree distance loss, the classifier will almost never predict leaf nodes of the hierarchy. 
Instead, it will often predict nodes close to the root of the hierarchy, which correspond to sets with many elements\,---\,a behavior that is unfavored if one wants to abstain only in cases of sufficient uncertainty. 

Quite obviously, methods that maximize set-based utility scores are closely connected to the quantification of uncertainty, since the decision about a suitable set of predictions is necessarily derived from information of that kind. The overwhelming majority of the above-mentioned methods depart from conditional class probabilities $p(y \given \vec{x}_q)$ that are estimated in a classical frequentist way, so that uncertainties in decisions are of aleatoric nature. 
Exceptions include {cite:p}`Yang2017` and {cite:p}`mpub385`, who further explore ideas from imprecise probability theory and reliable classification to generate label sets that capture both aleatoric and epistemic uncertainty.   

</div>
